In [9]:
import json
import numpy as np
import pandas as pd
pd.options.display.max_columns = None
import matplotlib.pyplot as plt

In [10]:
def check_avg(df_, season, team, stats = ['AST']):
    avg_stats = ['AVG_'+stat for stat in stats]
    df = df_.loc[(df_['SEASON_ID'] == season)& (df_['TEAM_ABBREVIATION'] == team)].sort_values('GAME_DATE')
    avg = df[stats].expanding().mean().shift(1)
    diff = df[avg_stats].values - avg[stats].values
    return(np.max(diff[1:]), np.min(diff[1:]))

In [11]:
STATS = ['PTS', 'FG2M', 'FG2A', 'FG2_PCT', 'FG3M', 'FG3A', 'FG3_PCT', 'FTM', 'FTA', 'FT_PCT', 'DREB', 'OREB', 'AST', 'STL', 'BLK', 'TOV', 'PF']
AVG_STATS = ['AVG_'+stat for stat in STATS]
OPP_AVG_STATS = ['OPP_'+stat for stat in AVG_STATS]

In [12]:
df = pd.read_csv('dat/1983-2017.csv', index_col=0).sort_values(['SEASON_ID', 'TEAM_ID', 'GAME_DATE'])
df = df.set_index('GAME_ID')
df['FG3A'] = df['FG3A'].fillna(0)
df.loc[df['FG3A']==0, ['FG3M', 'FG3_PCT']] = 0
df['FG2M'] = df['FGM'] - df['FG3M']
df['FG2A'] = df['FGA'] - df['FG3A']
df['FG2_PCT'] = df['FG2M'] / df['FG2A']
df['PLUS_MINUS'] = df['PTS'] - df['PTS_A']
df = df.reset_index()

In [13]:
for season in df.loc[df[STATS].isna().any(axis=1)]['SEASON_ID'].values:
    df = df.loc[df['SEASON_ID']!=season]

In [14]:
df[STATS].isna().any(axis=0)

PTS        False
FG2M       False
FG2A       False
FG2_PCT    False
FG3M       False
FG3A       False
FG3_PCT    False
FTM        False
FTA        False
FT_PCT     False
DREB       False
OREB       False
AST        False
STL        False
BLK        False
TOV        False
PF         False
dtype: bool

In [15]:
df.sample(10)

,GAME_ID,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_DATE,MATCHUP,WL,MIN,PTS,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PLUS_MINUS,PTS_A,FG2M,FG2A,FG2_PCT,WINS,PTS_AVG,PTS_A_AVG,TOTAL_GAMES,TOTAL_WINS,GP,PYE,PYE_WINS,PYE_WINS_ERROR,PYE_WINS_ERROR_ABS,PYE_PROJ,PYE_PROJ_ERROR,PYE_PROJ_ERROR_ABS,year
29350,29601116,21996,1610612764,WAS,Washington Bullets,1997-04-11,WAS vs. NJN,W,239,109,47,86,0.547,2,12.0,0.167,13,18,0.722,6.0,33.0,39.0,27,12.0,5.0,9.0,18.0,19,90,45,74.0,0.608108,40.0,99.051948,97.935065,82,44.0,77,0.539353,41.530146,-1.530146,1.530146,42.696763,1.303237,1.303237,1996-97
7047,28600899,21986,1610612757,POR,Portland Trail Blazers,1987-04-12,POR vs. LAC,W,240,113,47,91,0.516,0,0.0,0.000,19,25,0.760,13.0,38.0,51.0,34,14.0,3.0,23.0,25.0,13,100,47,91.0,0.516484,47.0,117.769231,114.230769,82,49.0,78,0.604522,47.152722,-0.152722,0.152722,49.418088,-0.418088,0.418088,1986-87
50035,20600219,22006,1610612753,ORL,Orlando Magic,2006-11-29,ORL @ SEA,W,240,94,34,67,0.507,2,7.0,0.286,24,33,0.727,11.0,29.0,40.0,21,7.0,8.0,17.0,18.0,10,84,32,60.0,0.533333,12.0,96.625000,92.125000,82,40.0,16,0.660020,10.560322,1.439678,1.439678,55.561329,-15.561329,15.561329,2006-07
70667,21500841,22015,1610612740,NOP,New Orleans Pelicans,2016-02-23,NOP @ WAS,L,241,89,29,83,0.349,6,23.0,0.261,25,33,0.758,8.0,40.0,48.0,17,8.0,4.0,16.0,17.0,-20,109,23,60.0,0.383333,22.0,102.375000,105.321429,82,30.0,56,0.402589,22.545009,-0.545009,0.545009,32.467325,-2.467325,2.467325,2015-16
53661,20800514,22008,1610612737,ATL,Atlanta Hawks,2009-01-07,ATL vs. ORL,L,240,102,40,84,0.476,8,21.0,0.381,14,25,0.560,9.0,30.0,39.0,27,12.0,5.0,13.0,23.0,-4,106,32,63.0,0.507937,22.0,98.294118,95.264706,82,47.0,34,0.607174,20.643924,1.356076,1.356076,51.144363,-4.144363,4.144363,2008-09
33122,29901107,21999,1610612752,NYK,New York Knicks,2000-04-10,NYK vs. IND,W,241,83,36,78,0.462,5,13.0,0.385,6,7,0.857,5.0,34.0,39.0,20,5.0,5.0,13.0,25.0,2,81,31,65.0,0.476923,48.0,92.363636,90.311688,82,50.0,77,0.577497,44.467303,3.532697,3.532697,50.887487,-0.887487,0.887487,1999-00
66696,21300324,22013,1610612752,NYK,New York Knicks,2013-12-11,NYK vs. CHI,W,241,83,31,76,0.408,5,19.0,0.263,16,19,0.842,10.0,29.0,39.0,17,16.0,7.0,13.0,21.0,5,78,26,57.0,0.456140,6.0,93.761905,97.714286,82,37.0,21,0.360238,7.564996,-1.564996,1.564996,27.974512,9.025488,9.025488,2013-14
4401,28500637,21985,1610612745,HOU,Houston Rockets,1986-02-21,HOU @ DAL,W,240,111,47,101,0.465,3,7.0,0.429,14,17,0.824,21.0,27.0,48.0,29,6.0,7.0,9.0,23.0,7,104,44,94.0,0.468085,36.0,114.750000,112.303571,82,51.0,56,0.574385,32.165537,3.834463,3.834463,50.934000,0.066000,0.066000,1985-86
38446,20101024,22001,1610612759,SAS,San Antonio Spurs,2002-03-27,SAS @ POR,L,240,93,38,70,0.543,5,16.0,0.313,12,20,0.600,8.0,29.0,37.0,25,5.0,4.0,16.0,24.0,-5,98,33,54.0,0.611111,49.0,97.239437,91.492958,82,58.0,71,0.700004,49.700310,-0.700310,0.700310,56.700048,1.299952,1.299952,2001-02
37708,20101027,22001,1610612750,MIN,Minnesota Timberwolves,2002-03-28,MIN vs. DAL,W,240,113,46,84,0.548,4,10.0,0.400,17,21,0.810,8.0,30.0,38.0,33,4.0,2.0,6.0,21.0,2,111,42,74.0,0.567568,43.0,99.901408,96.352113,82,50.0,71,0.623208,44.247764,-1.247764,1.247764,49.855287,0.144713,0.144713,2001-02


In [16]:
df[AVG_STATS] = df.sort_values(['SEASON_ID', 'TEAM_ID', 'GAME_DATE']).groupby(['SEASON_ID', 'TEAM_ID'])[STATS].expanding().mean().reset_index((0,1))[STATS]
df[AVG_STATS] = df.sort_values(['SEASON_ID','TEAM_ID', 'GAME_DATE']).groupby(['SEASON_ID', 'TEAM_ID'])[AVG_STATS].shift(1)
df = df.sort_values(['SEASON_ID', 'TEAM_ID', 'GAME_DATE']).set_index('GAME_ID', drop=True)

In [17]:
df.head(10)[['PTS', 'AVG_PTS', 'AST', 'AVG_AST']]

,PTS,AVG_PTS,AST,AVG_AST
GAME_ID,,,,
28500005,91,NaN,25,NaN
28500013,91,91.000000,14,25.000000
28500030,102,91.000000,21,19.500000
28500045,105,94.666667,22,20.000000
28500054,114,97.250000,25,20.500000
28500060,113,100.600000,18,21.400000
28500069,114,102.666667,17,20.833333
28500076,119,104.285714,25,20.285714
28500085,97,106.125000,26,20.875000


In [18]:
df['perc_complete'] = (df['GP'].values-1) / df['TOTAL_GAMES']

In [19]:
home = df.loc[~df['MATCHUP'].str.contains('@')].sort_index()
away = df.loc[df['MATCHUP'].str.contains('@')].sort_index()

In [20]:
home['HOME'] = 1
away['HOME'] = 0

In [21]:
opp_headers = ['OPP_'+stat for stat in STATS+AVG_STATS]

In [22]:
away_opp = away.rename(columns={stat: 'OPP_'+stat for stat in STATS+AVG_STATS})
home_opp = home.rename(columns={stat: 'OPP_'+stat for stat in STATS+AVG_STATS})

In [23]:
data = pd.concat([pd.concat([home, away_opp[opp_headers]], axis=1), pd.concat([away, home_opp[opp_headers]], axis=1)])

In [24]:
data['W'] = data['WL'].map({'W': 1, 'L': 0})

In [25]:
with open('dat/locations.json') as f:
    locations = json.loads(f.read())
data['LOCATION'] = data['TEAM_NAME'].map(locations)
data['LOCATION_W_PCT'] = data['LOCATION'].map(data.loc[data['HOME'] == 1].groupby('LOCATION')['W'].mean().to_dict())

In [26]:
data = data.reset_index()

In [27]:
data.sort_values(['SEASON_ID', 'TEAM_ID', 'GAME_DATE']).head()[['GAME_DATE', 'TEAM_ABBREVIATION', 'MATCHUP', 'PTS', 'AVG_PTS', 'OPP_PTS', 'OPP_AVG_PTS']]

,GAME_DATE,TEAM_ABBREVIATION,MATCHUP,PTS,AVG_PTS,OPP_PTS,OPP_AVG_PTS
21739,1985-10-25,ATL,ATL vs. WAS,91,NaN,100,NaN
58734,1985-10-26,ATL,ATL @ MIL,91,91.000000,117,116.000000
21764,1985-10-29,ATL,ATL vs. NYK,102,91.000000,87,89.000000
58766,1985-11-01,ATL,ATL @ BOS,105,94.666667,109,110.333333
21788,1985-11-02,ATL,ATL vs. PHL,114,97.250000,113,112.500000


*NOTE: OPP_AVG_PTS is not average points scored against, it is AVG_PTS for other team in matchup*

In [35]:
AVG_STATS_A = [stat+'_A' for stat in AVG_STATS]
OPP_STATS = ['OPP_'+stat for stat in STATS]

In [37]:
data[AVG_STATS_A] = data.sort_values(['SEASON_ID', 'TEAM_ID', 'GAME_DATE']).groupby(['SEASON_ID', 'TEAM_ID'])[OPP_STATS].expanding().mean().reset_index((0,1))[OPP_STATS]
data[AVG_STATS_A] = data.sort_values(['SEASON_ID','TEAM_ID', 'GAME_DATE']).groupby(['SEASON_ID', 'TEAM_ID'])[AVG_STATS_A].shift(1)

In [41]:
OPP_AVG_STATS_A = [stat+'_A' for stat in OPP_AVG_STATS]

In [44]:
data = data.set_index('GAME_ID')
home = data.loc[data['HOME']==1].sort_index()
away = data.loc[data['HOME']==0].sort_index()

In [45]:
home[OPP_AVG_STATS_A] = away[AVG_STATS_A]
away[OPP_AVG_STATS_A] = home[AVG_STATS_A]

In [46]:
new = pd.concat([home, away])

In [59]:
phi_2017 = new.loc[(new['SEASON_ID']==22017) & (new['TEAM_ABBREVIATION']=='PHI')].sort_values('GAME_DATE')
phi_2017[['PTS', 'AVG_PTS', 'OPP_PTS', 'AVG_PTS_A']].head()

,PTS,AVG_PTS,OPP_PTS,AVG_PTS_A
GAME_ID,,,,
21700006,115,NaN,120,NaN
21700019,92,115.000000,102,120.000000
21700027,94,103.500000,128,111.000000
21700041,97,100.333333,86,116.666667
21700057,104,99.500000,105,109.000000


In [61]:
phi_2017[['MATCHUP', 'OPP_AVG_PTS', 'OPP_AVG_PTS_A']].head()

,MATCHUP,OPP_AVG_PTS,OPP_AVG_PTS_A
GAME_ID,,,
21700006,PHI @ WAS,NaN,NaN
21700019,PHI vs. BOS,99.5,105.0
21700027,PHI @ TOR,117.0,100.0
21700041,PHI @ DET,108.0,104.0
21700057,PHI vs. HOU,106.0,102.5


In [69]:
new.loc[21700091, ['TEAM_ABBREVIATION', 'MATCHUP', 'AVG_PTS', 'AVG_PTS_A', 'OPP_AVG_PTS', 'OPP_AVG_PTS_A']]

,TEAM_ABBREVIATION,MATCHUP,AVG_PTS,AVG_PTS_A,OPP_AVG_PTS,OPP_AVG_PTS_A
GAME_ID,,,,,,
21700091,GSW,GSW vs. DET,120.0,114.166667,104.5,99.500000
21700091,DET,DET @ GSW,104.5,99.500000,120.0,114.166667


In [70]:
new.to_csv('dat/expanding_avg_1983-2017.csv')